<a href="https://colab.research.google.com/github/Krystal-Jing/NLP_Course_Materials/blob/master/%E2%80%9Clab02_Embeddings_solutions_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic word representations

Semantic Word Representations are representations that are learned to capture the 'meaning' of a word based on the contexts it occurs in. These are low-dimensional vectors that contain some semantic properties. In this notebook we are going to build state-of-the art approaches to obtain semantic word representations using the `word2vec` modelling approach. We will use these vectors in some tasks to understand the utility of these representations. Later we will also learn how to use such already pre-trained representations (BERT) in those tasks.

We begin by loading some of the libraries that are necessary for building our model. We are using [pytorch](https://pytorch.org/), an open source deep learning platform, as our backbone library in the course.

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
from scipy.spatial.distance import euclidean, cosine
from tqdm import tqdm 
import codecs
from sklearn.metrics.pairwise import cosine_distances
import nltk
nltk.download('punkt')

In [ ]:
corpus = [
    'he is a king',
    'she is a queen',
    'he is a Man',
    'she Is a woman',
    'london is, the capital of England',
    'Berlin is ... the capital of germany',
    'paris is the capital of france.',
    'He will eat cake, pie, and/or brownies',
    "she didn't like the brownies"
]

We will reuse the pre-processing and vocabulary methods we have created in the previous lab.

In [ ]:
import string
import re
from nltk import word_tokenize

def preprocess_corpus(corpus):
  tokenized_corpus = [] 
  for sentence in corpus:
    sentence = " ".join(word_tokenize(sentence))
    tokenized_sentence = []
    for token in sentence.split(' '):
      token = token.lower()
      tokenized_sentence.append(token)
    tokenized_corpus.append(tokenized_sentence)
  return tokenized_corpus

def get_vocabulary(tokenized_corpus):
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
        vocabulary.append(token)
  return set(vocabulary)

tokenized_corpus = preprocess_corpus(corpus)
print(tokenized_corpus)

vocabulary = get_vocabulary(tokenized_corpus)
vocabulary_size = len(vocabulary)
print(vocabulary_size)




##Helper functions
These are some of other common helper functions that are used for NLP models:

`word2idx`: Maintains a dictionary of word and the corresponding index

`idx2word`: Maintains a mapping from index to word

Print the word2idx and idx2word, we will be using these in future exercises.

In [ ]:
word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

print(word2idx)
print(idx2word)

## Look-up table

This is a table that maps from an index to a one hot vector. It is a categorical variable binary representation where each row has one feature with a value of 1, and the other features with value 0. 1 in a particular column will tell the correct category for each example.

**Q. Print one-hot vectors corresponding to the words 'the', 'he' and ''england'**

*A. See code below*

In [ ]:
def look_up_table(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x
  
# This is a one hot representation

# Q. try printing it for word_idx = 1

word_idx = word2idx['he']
print(look_up_table(word_idx))

##Extracting contexts and the focus word

We will be building the **skip-gram** model. Given a corpus this model tries to use the current word (focus word) to predict its neighbors (its context)

We first begin by obtaining the set of contexts and focus words.

Let us say we have a sentence (represented as vocabulary indicies): [0, 2, 3, 6, 7].
For every word in the sentence, we want to get the words which are window_size around it.
So if window_size==2, for the word '0', we obtain: [[0, 2], [0, 3]]
For the word '2', we obtain: [[2, 0], [2, 3], [2, 6]]
For the word '3', we obtain: [[3, 0], [3, 2], [3, 6], [3, 7]]

**Q. Print some of the index pairs and trace them back to their words.**

In [ ]:
window_size = 2
idx_pairs = []

# variables of interest: 
#   center_word_pos: center word position
#   context_word_pos: context_word_position
#   add sentence length as a constraint

for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    
    for center_word_pos in range(len(indices)):
        
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
                
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

print(idx_pairs)

In [ ]:
# We'll sample 5 elements at random and trace these back to their word pairs
from random import sample

random_pairs = sample(list(idx_pairs), 5)
print(random_pairs)

tokens_from_idx_pairs = []
for random_pair in random_pairs:
    focus_word_idx, context_word_idx = random_pair[0], random_pair[1]
    focus_word = idx2word[focus_word_idx]
    context_word = idx2word[context_word_idx]
    tokens_from_idx_pairs.append([focus_word, context_word])

print()
print(tokens_from_idx_pairs)

## Parameters and hyperparameters¶
For our toy task, let us set the embedding dimensions to 5. Let us run the algorithm for 10 epochs (number of times the training algorithm sees all the training data). Let us choose the learning rate of 0.001. We have two parameter matrices $W_1$ and $W_2$ - the embedding matrix and the weight matrix.

**Q. What are the dimensionalities of $W_1$ and $W_2$?** 

*A. shape(W1) = [embedding_dims x vocabulary_size]
shape(W2) = [vocabulary_size x embedding_dims]*

 


In [ ]:
# Hyperparameters:
embedding_dims = 5
num_epochs = 100
learning_rate = 0.001

# The two weight matrices:
W1 = torch.randn(embedding_dims, vocabulary_size, requires_grad=True)
W2 = torch.randn(vocabulary_size, embedding_dims, requires_grad=True)




##Training the model

In the code below, we are going to compute the log probability of the correct context (target) given the word.

**Q. Fill in the gaps.**
 
**Q. Print the loss and see if the loss goes down.**


**Q. What is the meaning of the negative log-likelihood loss (NLL) ?**

The better the prediction the lower the NLL loss. Minimising the negative likelihood is the same as maximising the average log probability for the context window. We do it to be consistent with the rule of loss functions. Log is used for numerical stability. Likelihood refers to the chances that our outputs produce the target distribution.

**Q. Which of the weight matrices will be extracted as word vectors ?**

A. W1

In [ ]:
 for epoch in tqdm(range(num_epochs)):
  
    loss_val = 0
    
    for data, target in idx_pairs:
      
        x = torch.Tensor(look_up_table(data))

        # Q. what would y_true be? 
        y_true = torch.Tensor([target]).long()

        # A. [index] of the target word
        # 
        z1 = torch.matmul(W1, x) 
        
        z2 = torch.matmul(W2, z1)
        # Q. what is the above operation?
        # A. matrix multiplication
 
        # Let us obtain prediction over the vocabulary
        log_softmax = F.log_softmax(z2, dim=0)
                
        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.item()
        
        # propagate the error
        loss.backward()
        
        # gradient descent
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        # zero out gradient accumulation
        W1.grad.data.zero_()
        W2.grad.data.zero_()

    print(f'\nFinal epoch loss: {loss_val/len(idx_pairs)}')

**Q. Given that we are interested in distributed representations, what is the major bottleneck in our setup? Is it the dimensionality of the representations? Is it the learning rate? Is it the corpus?**

*A. It is both the dimensionality and the corpus. We have a only few words and contexts it would be difficult to capture distributional contexts. As we increase the words or expand the corpus we would have to expand our dimensions.*

**Q. What hyperparameters would you tune to improve the representations?**

*A. Decreasing the learning rate (this will be discussed more in further lectures)*

## Using word embeddings

One of the simplest ways of exploiting word representations is to find similar words. There are many ways of measuring the semantic similarity between two words. As we are using word representations which are vectors in the euclidean space, distance metrics defined in the euclidean space are the most popular choice. This is because words that share common contexts in the corpus are located in close proximity to one another in the euclidean space. One such metric is the eucldeian distance.

**Q. What is the euclidean distance between 'the' and 'a' (in the sample corpus and the new corpus)?**

*A. See solution in code.*

**Q. What other distance metrics can we use for two vectors?**

A. Cosine distance is the most frequently used distance metric for high-dimensional space. Many other distance metrics exist and high-level overview can be found in [this blog post](https://towardsdatascience.com/importance-of-distance-metrics-in-machine-learning-modelling-e51395ffe60d)

In [ ]:
# Let us get two vectors from the trained model

x = torch.Tensor(look_up_table(0))
x_emb = torch.matmul(W1, x).detach().numpy()
y = torch.Tensor(look_up_table(1))
y_emb = torch.matmul(W1, y).detach().numpy()

# let us print the euclidean distance
print(euclidean(x_emb, y_emb))

In [ ]:
vector_the = look_up_table(word2idx["the"])
vector_a = look_up_table(word2idx["a"])
print(euclidean(vector_the, vector_a))


# Pre-trained representations

We have seen from the above that word embeddings are learned in an unsupervised manner, i.e., we don't have any labelled data. These representations can be used to `bootstrap' models in NLP. There are many word representation inducing algorithms : word2vec, GloVe, Fasttext are some of the popular choices. There are differences in the algorithms but they are all based on the distributional hypothesis (a word could be defined by its distribution in language use).

We will now use one of these pre-trained representations: [GloVe](https://nlp.stanford.edu/projects/glove/).

**Q. What is the dimensionality of the representations below?**

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
w2i = [] # word2index
i2w = [] # index2word
wvecs = [] # word vectors

# this is a large file, it will take a while to load in the memory!
with codecs.open('glove.6B.50d.txt', 'r','utf-8') as f: 
  index = 0
  for line in tqdm(f.readlines()):
    # Ignore the first line - first line typically contains vocab, dimensionality
    if len(line.strip().split()) > 3:
      
      (word, vec) = (line.strip().split()[0], 
                     list(map(float,line.strip().split()[1:]))) 
      
      wvecs.append(vec)
      w2i.append((word, index))
      i2w.append((index, word))
      index += 1

w2i = dict(w2i)
i2w = dict(i2w)
wvecs = np.array(wvecs)

For the following experiments, we recommend using `wvecs` - the pre-trained representations.

## Evaluating word representation models
 
Evaluation of word representations is done in specific subtasks.

### Word Similarity
The first task we consider is evaluating if the representations are good at computing if two words are similar in meaning. 

### Exploring Analogies
The second task we consider is completing analogies. We are given an incomplete analogy of the form:

$a : b : : c :~?$

We then identify the word vector which maximises the cosine similarity. Ideally, we want $\phi(b) - \phi(a) = \phi(d) - \phi(c)$ where $\phi(.)$ is the word vector. For example, london $-$ england = paris $-$ france .


Use the examples below to answer the following questions:

**Q. When does the analogy task it fail?**

*for more rare words, for grammatical relations*

**Q. What are the possible reasons for failure?**

*word is rare, context more ambiguos. For more explanations, refer to this [blog](https://kawine.github.io/blog/nlp/2019/06/21/word-analogies.html*


In [ ]:
def cosine_distance(u, v):
    distance = 0.0
    dot = np.dot(u,v)
    norm_u = np.sqrt(np.sum(u**2))
    norm_v = np.sqrt(np.sum(v**2))
    distance = dot/(norm_u)/norm_v
    return distance
  
 
def find_analogy(word_a, word_b, word_c, word_vectors, word2index):
    word_a = word_a.lower()
    word_b = word_b.lower()
    word_c = word_c.lower()
    
    (e_a, e_b, e_c) = (word_vectors[word2index[word_a]], 
                       word_vectors[word2index[word_b]], 
                       word_vectors[word2index[word_c]])
    
    
    max_cosine_sim = -999
    best_word = None
    
    for (w, i) in word2index.items():
        if w in [word_a, word_b, word_c]:
            continue
        cosine_sim = cosine_distance(e_b - e_a, word_vectors[i] - e_c)
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
            
    return best_word
 
#correct answer: France 
print(find_analogy('athens', 'greece', 'paris', wvecs, w2i))
#correct answer: she
print(find_analogy('stepson', 'stepdaughter', 'he', wvecs, w2i))
#correct answer: Tuvalu
print(find_analogy('bangkok', 'thailand', 'funafuti', wvecs, w2i))
#correct answer: cars
print(find_analogy('woman', 'women', 'car', wvecs, w2i))
#write writes vanish vanishes
print(find_analogy('write', 'writes', 'vanish', wvecs, w2i))

## Pre-trained contextualised representations (BERT)

We will now use one of the recently most popular pre-trained representations: BERT (Bidirectional Encoder Representations from Transformers). BERT creates so called contextualised word embeddings. You will see the details of creating such representations later in the course. 

For now, you need to know that BERT produces word representations which are informed by immediate sentence context rather than the the whole corpus (like for `word2vec`). For example, we have two sentences: *The man went to the bank to open an account* and *The man was sitting by the bank of the river*. `word2vec` will give the same word embedding for the word *bank* in both sentences, while BERT embeddings for *bank* would be different for each sentence since the meaning of this word is different. We will see it later in the lab.

We use the Pytorch interface for BERT by [Hugging Face](https://github.com/huggingface/transformers).

In [ ]:
!pip install transformers

We will use the model released by Google. It is a small `base` model which ignores casing (*uncased*). BERT uses a specific preprocessing procedure, which is provided by the BertTokenizer.

**Q. Why do we need to tokenise our inputs according to BERT rules?**

*A. So that words in our inputs are recognised by the model and are matched to repective pre-trained representations.*

In [ ]:
from transformers import BertTokenizer, BertModel
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


BERT can take as input one or two sentences. In this tutorial, we will focus on one sentence as input. It should be surrounded by the special tokens `[CLS]` and `[SEP]`. You will understand the details of this pre-processing later in the course.
 

In [ ]:
#input_sentence = corpus[7]
# uncomment and test for another sentence
input_sentence = 'After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank.'
marked_input_sentence = "[CLS] " + input_sentence + " [SEP]"

# tokenize our sentence with the BERT tokenizer
tokenized_text = tokenizer.tokenize(marked_input_sentence)

# print out the tokens
print (tokenized_text)

# map the token strings to their vocabulary indeces
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

Note that some words have been split into subwords: for example, 'brownies' into 'brown' and '##ies'. BERT uses the WordPiece model. WordPiece is similar to BPE with the main difference that it forms a new subword by likelihood but not the next pair with highest frequency.

As mentioned before, BERT can take up to two sentences in input. We need to specify which sentence our input belongs to. Single-sentence inputs only require a series of 1. If you want to process two sentences, we will assign each word in the first sentence a 0, and all tokens of the second sentence a 1.

In [ ]:
# mark each of the tokens as belonging to sentence 1
segments_ids = [1] * len(tokenized_text)

# convert our data to torch tensors and call the BERT model
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

print (segments_ids)

We will now load the `bert-base-uncased`. We will see the definition of the model printed in the logging. The model is a deep neural network with 12 layers. The meaning of these layers again will be explained later in the course. 

`model.eval()` puts our model in evaluation mode as opposed to training mode. This switches off certain regularisation techniques that modify input.
 


In [ ]:
# load the pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

# put the model into evaluation mode
model.eval()


`torch.no_grad` tells PyTorch not to construct the compute graph during this forward pass. We are not running training and will not backpropagate.

In [ ]:
 with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)
    print(outputs[1].size())
    # we take our first example
    word_vectors = outputs[0]
    print(word_vectors.size())
    # we do not need the batch dimension here since we have only one sentence
    word_vectors = torch.squeeze(word_vectors, dim=0)
    print(word_vectors.size())

Print out indices of the three instances of the word *bank* in the example sentence.

In [ ]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

In [ ]:
print('First 5 vector values for each instance of "bank".')
print('')
print("bank vault   ", str(word_vectors[6]))
print("bank robber  ", str(word_vectors[10]))
print("river bank   ", str(word_vectors[19]))


**Q. Calculate the cosine similarity between the word *bank* in (a) *bank robber* vs *river bank* (different meanings); (b) *bank robber* vs *bank vault* (same meaning). What are your conclusions?**

*A. Similarity between instances of *bank* with similar meaning is higher.*

In [ ]:
# "bank robber" vs "river bank" (different meanings)
diff_bank = cosine_distance(np.array(word_vectors[10]), np.array(word_vectors[19]))

# in "bank robber" vs "bank vault" (same meaning)
same_bank = cosine_distance(np.array(word_vectors[10]), np.array(word_vectors[6]))

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

## Advanced: Compositionality

**Q. Given access to only word representations, how can we build representations for phrases and sentences?**

*A. averaging, element-wise sum, concatenation of the embeddings for composing words*

**Q. Compute the sentence vectors for the two sentences below. What is the problem with this way to create sentence representations?**

*A. The word order is ignored.*

**Q. Would averaging BERT embeddings help?**

*A. Yes, the similarity is slightly lower.*


In [ ]:
sent1 = 'it was really not good , on the opposite quite bad .'
sent2 = 'it was really not bad , on the opposite quite good .'

def get_sent_embedding(sent):
  sent_embedding = []
  for word in sent.split(' '):
    sent_embedding.append(wvecs[w2i[word]])
  sent_embedding = np.mean(np.array(sent_embedding), axis=0)
  return sent_embedding

sent1_embedding = get_sent_embedding(sent1)
sent2_embedding = get_sent_embedding(sent2)

print(cosine_distance(sent1_embedding, sent2_embedding))


In [ ]:
def preprocess_for_bert(sent):
  marked_input_sentence = "[CLS] " + sent + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_input_sentence)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  return tokens_tensor, segments_tensors

sent1_tokens, sent1_segments = preprocess_for_bert(sent1)
sent2_tokens, sent2_segments = preprocess_for_bert(sent2)


with torch.no_grad():

    outputs_sent1 = model(sent1_tokens, sent1_segments)
    word_vectors_sent1 = outputs_sent1[0]
    word_vectors_sent1 = torch.squeeze(word_vectors_sent1, dim=0)
    sent1_embedding = word_vectors_sent1.mean(dim=0)
    sent1_embedding = sent1_embedding.detach().cpu().numpy()

    outputs_sent2 = model(sent2_tokens, sent2_segments)
    word_vectors_sent2 = outputs_sent2[0]
    word_vectors_sent2 = torch.squeeze(word_vectors_sent2, dim=0)
    sent2_embedding = word_vectors_sent2.mean(dim=0)
    sent2_embedding = sent2_embedding.detach().cpu().numpy()


print(cosine_distance(sent1_embedding, sent2_embedding))
 
  

# Assignment 1: Training with negative sampling

Fill in the gaps in the code below for training the skipgram model with negative sampling. Train the model using the following corpus:  https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt. Use can use the GPU Colab Runtime. Submit the code and a short report (150 words) answering the following questions:  

1. How did you preprocess your dataset and why?
2. How did you pick negative examples? How many of them did you choose? Why did you decide to do so?
3. How did you compute the final loss? 








In [ ]:
# The two weight matrices:
W1 = torch.randn(embedding_dims, vocabulary_size, requires_grad=True)
W2 = torch.randn(embedding_dims, vocabulary_size, requires_grad=True)

for epoch in range(num_epochs):
    epoch_loss = 0
    for data, target in idx_pairs:
        x_var = Variable(look_up_table(data)).float() 
        
        y_pos = Variable(torch.from_numpy(np.array([target])).long())
        y_pos_var = Variable(look_up_table(target)).float()

        # TO DO: pick a negative sample
        neg_sample = 
        
        #TO DO: use the look up table
         
        x_emb = torch.matmul(W1, x_var) 
        y_pos_emb = torch.matmul(W2, y_pos_var)
        y_neg_emb = torch.matmul(W2, y_neg_var)
        
        # get positive sample score
        pos_loss = F.logsigmoid(torch.matmul(x_emb, y_pos_emb))
        
        # get negsample score
        neg_loss = F.logsigmoid(-1 * torch.matmul(x_emb, y_neg_emb))
        
        # TO DO: compute loss
        loss =  
        epoch_loss += loss.item()
        
        # propagate the error
        loss.backward()
        
        # gradient descent
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        # zero out gradient accumulation
        W1.grad.data.zero_()
        W2.grad.data.zero_()
        
    if epoch % 10 == 0:    
        print(f'Loss at epoch {epoch}: {epoch_loss/len(idx_pairs)}')